# Day6 - Removing Outliers 1 (actually was a wrong way)

Yesterday, I tried stacking method, but I think the way I tuned hyperparameters were not optimal and I did not get a good score. 

Today, I am going to try removing outliers.

In [1]:
from sklearn import datasets
from sklearn.metrics import accuracy_score, f1_score, roc_curve, auc
from sklearn.model_selection import train_test_split, KFold
from sklearn.svm import LinearSVC, SVC
import numpy as np
from sklearn.metrics import mean_squared_error, log_loss
from sklearn.model_selection import GridSearchCV
import pandas as pd
from tqdm import tqdm
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv('train_final.csv')
X_pred_data = pd.read_csv('test_final.csv')
X_pred = np.asarray(X_pred_data.iloc[:, 1:25]).reshape(-1, 24)

In [3]:
X = np.asarray(data.iloc[:, 2:26]).reshape(-1, 24)
y = np.asarray(data.iloc[:, 1]).reshape(-1, 1)

In [4]:
mean = []
std = []
for i in range(X.shape[1]):
    mean.append(np.mean(X[:, i]))
    std.append(np.std(X[:, i]))

In [ ]:
plt.plot(mean)

In [ ]:
plt.plot(std)

Apparently, feature 23 has a bad sample. Actually I already found it. It's sample 4664, value is 404288627. This is eroding the quality of model.

Now I have to find a way to remove that row.

In [ ]:
for i in range(X.shape[1]):
    plt.plot(X[:, i])
    plt.title('feature' + str(i+1) + "   $\mu$ : %.1f     $\sigma$ : %.1f" %(mean[i], std[i]))
    plt.show()

In [5]:
num_outliers = []
for i in range(X.shape[1]):
    count = 0
    for j in range(X.shape[0]):
        if(X[j][i] > mean[i] + 3.5*std[i] or X[j][i] < mean[i] - 3.5*std[i]):
            count += 1
    num_outliers.append(count*100/X.shape[0])

In [6]:
for i in range(X.shape[1]):
    print("feature " + str(i+1) + "     outliers :    %.2f " %num_outliers[i] + "%")

feature 1     outliers :    0.70 %
feature 2     outliers :    3.52 %
feature 3     outliers :    0.08 %
feature 4     outliers :    0.42 %
feature 5     outliers :    3.59 %
feature 6     outliers :    0.71 %
feature 7     outliers :    0.71 %
feature 8     outliers :    0.00 %
feature 9     outliers :    3.32 %
feature 10     outliers :    0.28 %
feature 11     outliers :    3.46 %
feature 12     outliers :    0.13 %
feature 13     outliers :    0.70 %
feature 14     outliers :    0.20 %
feature 15     outliers :    0.96 %
feature 16     outliers :    2.33 %
feature 17     outliers :    0.00 %
feature 18     outliers :    0.61 %
feature 19     outliers :    3.27 %
feature 20     outliers :    3.55 %
feature 21     outliers :    3.71 %
feature 22     outliers :    3.46 %
feature 23     outliers :    0.03 %
feature 24     outliers :    3.57 %


Above data is showing how much portion of samples are out of 3.5$\sigma$ (i.e. standard deviation) range, if assumed to benormal distribution.

In normal distribution, it can be computed that $P(X>3.5\sigma\ or\ X<-3.5\sigma) = 0.00046\  or\  0.046%$. So, by comparing above values with this 0.046, I can get some idea if the feature came from a normal distribution or not. As below.


In [7]:
for i in range(X.shape[1]):
    if(num_outliers[i] <= 0.046): # 0.135
        print("feature " + str(i+1) + "     outliers :    %.3f " %num_outliers[i] + "%.  ->  Normal Distribution")
    else:
        print("feature " + str(i+1) + "     outliers :    %.3f " %num_outliers[i] + "%.  ->  Not")

feature 1     outliers :    0.696 %.  ->  Not
feature 2     outliers :    3.516 %.  ->  Not
feature 3     outliers :    0.079 %.  ->  Not
feature 4     outliers :    0.421 %.  ->  Not
feature 5     outliers :    3.589 %.  ->  Not
feature 6     outliers :    0.708 %.  ->  Not
feature 7     outliers :    0.708 %.  ->  Not
feature 8     outliers :    0.000 %.  ->  Normal Distribution
feature 9     outliers :    3.321 %.  ->  Not
feature 10     outliers :    0.281 %.  ->  Not
feature 11     outliers :    3.461 %.  ->  Not
feature 12     outliers :    0.134 %.  ->  Not
feature 13     outliers :    0.702 %.  ->  Not
feature 14     outliers :    0.201 %.  ->  Not
feature 15     outliers :    0.958 %.  ->  Not
feature 16     outliers :    2.332 %.  ->  Not
feature 17     outliers :    0.000 %.  ->  Normal Distribution
feature 18     outliers :    0.610 %.  ->  Not
feature 19     outliers :    3.272 %.  ->  Not
feature 20     outliers :    3.552 %.  ->  Not
feature 21     outliers :    3.711 %.

Thus, it turned out that features 8, 17, 23 are normal distribution.

Now, I will remove "outliers" for these 3 columns. For other features, I can't remove outliers so easily as they might not be normally distributed.

In [8]:
for i in [7, 16, 22]:
    j = 0
    while (j < X.shape[0]):
        if(X[j][i] > mean[i] + 3.5*std[i] or X[j][i] < mean[i] - 3.5*std[i]):
            val = X[j][i]
            X = np.delete(X, j, 0)
            y = np.delete(y, j, 0)
            print("because of feature %d, deleted row %d.    Deleted value : %.3f" %(i+1, j, val))
        else:
            j += 1

because of feature 23, deleted row 645.    Deleted value : 17681416.000
because of feature 23, deleted row 2995.    Deleted value : 42640756.000
because of feature 23, deleted row 4661.    Deleted value : 404288627.000
because of feature 23, deleted row 6044.    Deleted value : 17241070.000
because of feature 23, deleted row 8739.    Deleted value : 12860243.000


Now, I have deleted 5 samples which are pretty bad. I think they corrupted the data, and caused lower score.

In [ ]:
xgb0 = xgb.XGBClassifier(colsample_bytree=0.65, gamma=0.1,
       learning_rate=0.05, max_depth=5,
       min_child_weight=5, n_estimators=1000, objective='binary:logistic',
      reg_alpha=1, reg_lambda=1,
      subsample=0.85, verbosity=1, n_jobs=-1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [ ]:
xgb0.fit(X_train, y_train.ravel())

In [ ]:
print('Accuracy: %f' % metrics.accuracy_score(y_test, xgb.predict(X_test)))
print('AUC: %f' %metrics.roc_auc_score(y_test, xgb.predict(X_test)))

In [ ]:
y_submission20 = pd.DataFrame(xgb.predict_proba(X_pred)[:, 1], columns = ['Y']) 
y_submission20['Id'] = X_pred_data['Id']
y_submission20 = y_submission20.reindex(columns=["Id", "Y"])
y_submission20.to_csv("submission20_uk734.csv", index=False)

Now, I think I can try to remove more outliers, by trying $X>3\sigma\ or\ X<-3\sigma$.

In [9]:
for i in range(X.shape[1]):
    if(num_outliers[i] <= 0.27): 
        print("feature " + str(i+1) + "     outliers :    %.3f " %num_outliers[i] + "%.  ->  Normal Distribution")
    else:
        print("feature " + str(i+1) + "     outliers :    %.3f " %num_outliers[i] + "%.  ->  Not")

feature 1     outliers :    0.696 %.  ->  Not
feature 2     outliers :    3.516 %.  ->  Not
feature 3     outliers :    0.079 %.  ->  Normal Distribution
feature 4     outliers :    0.421 %.  ->  Not
feature 5     outliers :    3.589 %.  ->  Not
feature 6     outliers :    0.708 %.  ->  Not
feature 7     outliers :    0.708 %.  ->  Not
feature 8     outliers :    0.000 %.  ->  Normal Distribution
feature 9     outliers :    3.321 %.  ->  Not
feature 10     outliers :    0.281 %.  ->  Not
feature 11     outliers :    3.461 %.  ->  Not
feature 12     outliers :    0.134 %.  ->  Normal Distribution
feature 13     outliers :    0.702 %.  ->  Not
feature 14     outliers :    0.201 %.  ->  Normal Distribution
feature 15     outliers :    0.958 %.  ->  Not
feature 16     outliers :    2.332 %.  ->  Not
feature 17     outliers :    0.000 %.  ->  Normal Distribution
feature 18     outliers :    0.610 %.  ->  Not
feature 19     outliers :    3.272 %.  ->  Not
feature 20     outliers :    3.552 %

This is a little looser judgement. Feature 3, 12, 14 are "somewhat close to normal" distribution. Now I remove outliers again.

In [10]:
for i in [7, 16, 22, 2, 11, 13]:
    j = 0
    while (j < X.shape[0]):
        if(X[j][i] > mean[i] + 3.0*std[i] or X[j][i] < mean[i] - 3.0*std[i]):
            val = X[j][i]
            X = np.delete(X, j, 0)
            y = np.delete(y, j, 0)
            print("because of feature %d, deleted row %d.    Deleted value : %.3f" %(i+1, j, val))
        else:
            j += 1

because of feature 3, deleted row 1346.    Deleted value : 1283.250
because of feature 3, deleted row 2617.    Deleted value : 4060.380
because of feature 3, deleted row 3108.    Deleted value : 2463.840
because of feature 3, deleted row 3261.    Deleted value : 1446.090
because of feature 3, deleted row 4550.    Deleted value : 1805.400
because of feature 3, deleted row 5016.    Deleted value : 1290.330
because of feature 3, deleted row 6409.    Deleted value : 7129.560
because of feature 3, deleted row 7164.    Deleted value : 2030.190
because of feature 3, deleted row 7957.    Deleted value : 2318.700
because of feature 3, deleted row 8618.    Deleted value : 2070.900
because of feature 3, deleted row 12807.    Deleted value : 43910.160
because of feature 3, deleted row 14062.    Deleted value : 1552.290
because of feature 3, deleted row 14218.    Deleted value : 1761.150
because of feature 3, deleted row 15299.    Deleted value : 1095.630
because of feature 12, deleted row 693.    

because of feature 14, deleted row 11712.    Deleted value : -5.410
because of feature 14, deleted row 11757.    Deleted value : -5.018
because of feature 14, deleted row 11821.    Deleted value : 5.730
because of feature 14, deleted row 11846.    Deleted value : -5.029
because of feature 14, deleted row 12314.    Deleted value : -5.451
because of feature 14, deleted row 12334.    Deleted value : -5.974
because of feature 14, deleted row 12426.    Deleted value : -4.968
because of feature 14, deleted row 13089.    Deleted value : -5.834
because of feature 14, deleted row 13487.    Deleted value : 5.909
because of feature 14, deleted row 13722.    Deleted value : -4.952
because of feature 14, deleted row 13839.    Deleted value : -5.104
because of feature 14, deleted row 14004.    Deleted value : -5.103
because of feature 14, deleted row 14088.    Deleted value : 5.682
because of feature 14, deleted row 14108.    Deleted value : 6.339
because of feature 14, deleted row 14451.    Deleted

In [11]:
X.shape

(16206, 24)

## Below, I was just re-tuning XGBClassifier with the improved data. However, the way I removed outliers were wrong so it's not anything valid. (commented when creating the report 10/31).

In [ ]:
xgb01 = xgb.XGBClassifier()

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [ ]:
%time xgb01.fit(X_train, y_train.ravel())

In [ ]:
print('Accuracy: %f' % metrics.accuracy_score(y_test, xgb2.predict(X_test)))
print('AUC: %f' %metrics.roc_auc_score(y_test, xgb2.predict(X_test)))

In [ ]:
y_submission21 = pd.DataFrame(xgb2.predict_proba(X_pred)[:, 1], columns = ['Y']) 
y_submission21['Id'] = X_pred_data['Id']
y_submission21 = y_submission21.reindex(columns=["Id", "Y"])
y_submission21.to_csv("submission21_uk734.csv", index=False)

In [13]:
params1 = [
 {'max_depth':[5, 6],
 'min_child_weight':[5, 6],
 'gamma':[0.1],
 'subsample':[0.8],
 'colsample_bytree':[0.6],
 'reg_alpha':[1], 
 'objective': ['binary:logistic'],
 'scale_pos_weight':[1],
 'learning_rate': [0.1],
 'n_estimators': [1000]}
]

In [14]:
grid1 = GridSearchCV(xgb.XGBClassifier(), param_grid = params1, scoring='roc_auc', n_jobs=-1, iid=False, cv=3, verbose = 2)
grid1.fit(X_train, y_train.ravel())

Fitting 3 folds for each of 4 candidates, totalling 12 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:  2.4min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:  2.4min finished


GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=3, min_child_weight=1, missing=None,
       n_estimators=100, n_jobs=1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
       subsample=1, verbosity=1),
       fit_params=None, iid=False, n_jobs=-1,
       param_grid=[{'max_depth': [5, 6], 'min_child_weight': [5, 6], 'gamma': [0.1], 'subsample': [0.8], 'colsample_bytree': [0.6], 'reg_alpha': [1], 'objective': ['binary:logistic'], 'scale_pos_weight': [1], 'learning_rate': [0.1], 'n_estimators': [1000]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=2)

In [15]:
    xgb1 = grid1.best_estimator_
print(xgb1.get_params()['max_depth'])
print(xgb1.get_params()['min_child_weight'])

6
5


In [16]:
params2 = [
 {'max_depth':[6, 7],
 'min_child_weight':[4, 5],
 'gamma':[0.1],
 'subsample':[0.8],
 'colsample_bytree':[0.6],
 'reg_alpha':[1], 
 'objective': ['binary:logistic'],
 'scale_pos_weight':[1],
 'learning_rate': [0.1],
 'n_estimators': [1000]}
]

In [17]:
grid2 = GridSearchCV(xgb.XGBClassifier(), param_grid = params2, scoring='roc_auc', n_jobs=-1, iid=False, cv=3, verbose = 2)
grid2.fit(X_train, y_train.ravel())

Fitting 3 folds for each of 4 candidates, totalling 12 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:  2.6min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:  2.6min finished


GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=3, min_child_weight=1, missing=None,
       n_estimators=100, n_jobs=1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
       subsample=1, verbosity=1),
       fit_params=None, iid=False, n_jobs=-1,
       param_grid=[{'max_depth': [6, 7], 'min_child_weight': [4, 5], 'gamma': [0.1], 'subsample': [0.8], 'colsample_bytree': [0.6], 'reg_alpha': [1], 'objective': ['binary:logistic'], 'scale_pos_weight': [1], 'learning_rate': [0.1], 'n_estimators': [1000]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=2)

In [18]:
xgb2 = grid2.best_estimator_
print(xgb2.get_params()['max_depth'])
print(xgb2.get_params()['min_child_weight'])

6
4


In [19]:
params3 = [
 {'max_depth':[6],
 'min_child_weight':[3, 4],
 'gamma':[0.1],
 'subsample':[0.8],
 'colsample_bytree':[0.6],
 'reg_alpha':[1], 
 'objective': ['binary:logistic'],
 'scale_pos_weight':[1],
 'learning_rate': [0.1],
 'n_estimators': [1000]}
]

In [20]:
grid3 = GridSearchCV(xgb.XGBClassifier(), param_grid = params3, scoring='roc_auc', n_jobs=-1, iid=False, cv=3, verbose = 2)
grid3.fit(X_train, y_train.ravel())

Fitting 3 folds for each of 2 candidates, totalling 6 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   6 | elapsed:   57.4s remaining:   57.4s
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:  1.5min finished


GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=3, min_child_weight=1, missing=None,
       n_estimators=100, n_jobs=1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
       subsample=1, verbosity=1),
       fit_params=None, iid=False, n_jobs=-1,
       param_grid=[{'max_depth': [6], 'min_child_weight': [3, 4], 'gamma': [0.1], 'subsample': [0.8], 'colsample_bytree': [0.6], 'reg_alpha': [1], 'objective': ['binary:logistic'], 'scale_pos_weight': [1], 'learning_rate': [0.1], 'n_estimators': [1000]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=2)

In [21]:
xgb3 = grid3.best_estimator_
print(xgb3.get_params()['max_depth'])
print(xgb3.get_params()['min_child_weight'])

6
4


In [23]:
params4 = [
 {'max_depth':[6],
 'min_child_weight':[4],
 'gamma':[0, 0.1, 0.2],
 'subsample':[0.8],
 'colsample_bytree':[0.6],
 'reg_alpha':[1], 
 'objective': ['binary:logistic'],
 'scale_pos_weight':[1],
 'learning_rate': [0.1],
 'n_estimators': [1000]}
]

In [24]:
grid4 = GridSearchCV(xgb.XGBClassifier(), param_grid = params4, scoring='roc_auc', n_jobs=-1, iid=False, cv=3, verbose = 2)
grid4.fit(X_train, y_train.ravel())

Fitting 3 folds for each of 3 candidates, totalling 9 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   7 out of   9 | elapsed:  1.7min remaining:   29.7s
[Parallel(n_jobs=-1)]: Done   9 out of   9 | elapsed:  2.2min finished


GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=3, min_child_weight=1, missing=None,
       n_estimators=100, n_jobs=1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
       subsample=1, verbosity=1),
       fit_params=None, iid=False, n_jobs=-1,
       param_grid=[{'max_depth': [6], 'min_child_weight': [4], 'gamma': [0, 0.1, 0.2], 'subsample': [0.8], 'colsample_bytree': [0.6], 'reg_alpha': [1], 'objective': ['binary:logistic'], 'scale_pos_weight': [1], 'learning_rate': [0.1], 'n_estimators': [1000]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=2)

In [25]:
xgb4 = grid4.best_estimator_
print(xgb4.get_params()['gamma'])

0.1


In [26]:
xgb4.fit(X_train, y_train.ravel())

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=0.6, gamma=0.1,
       learning_rate=0.1, max_delta_step=0, max_depth=6,
       min_child_weight=4, missing=None, n_estimators=1000, n_jobs=1,
       nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=1, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=None, subsample=0.8, verbosity=1)

In [27]:
print('Accuracy: %f' % metrics.accuracy_score(y_test, xgb4.predict(X_test)))
print('AUC: %f' %metrics.roc_auc_score(y_test, xgb4.predict(X_test)))

Accuracy: 0.965475
AUC: 0.737617


This is a great improvement. I should try submitting at this point. I may achieve better score by further tuning it model.

In [28]:
y_submission22 = pd.DataFrame(xgb4.predict_proba(X_pred)[:, 1], columns = ['Y']) 
y_submission22['Id'] = X_pred_data['Id']
y_submission22 = y_submission22.reindex(columns=["Id", "Y"])
y_submission22.to_csv("submission22_uk734.csv", index=False)

In [29]:
params5 = [
 {'max_depth':[6],
 'min_child_weight':[4],
 'gamma':[0.1],
 'subsample':[0.7, 0.8],
 'colsample_bytree':[0.6, 0.7],
 'reg_alpha':[1], 
 'objective': ['binary:logistic'],
 'scale_pos_weight':[1],
 'learning_rate': [0.1],
 'n_estimators': [1000]}
]

In [30]:
grid5 = GridSearchCV(xgb.XGBClassifier(), param_grid = params5, scoring='roc_auc', n_jobs=-1, iid=False, cv=3, verbose = 2)
grid5.fit(X_train, y_train.ravel())

Fitting 3 folds for each of 4 candidates, totalling 12 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:  2.6min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:  2.6min finished


GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=3, min_child_weight=1, missing=None,
       n_estimators=100, n_jobs=1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
       subsample=1, verbosity=1),
       fit_params=None, iid=False, n_jobs=-1,
       param_grid=[{'max_depth': [6], 'min_child_weight': [4], 'gamma': [0.1], 'subsample': [0.7, 0.8], 'colsample_bytree': [0.6, 0.7], 'reg_alpha': [1], 'objective': ['binary:logistic'], 'scale_pos_weight': [1], 'learning_rate': [0.1], 'n_estimators': [1000]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=2)

In [31]:
xgb5 = grid5.best_estimator_
print(xgb5.get_params()['subsample'])
print(xgb5.get_params()['colsample_bytree'])

0.8
0.6


In [32]:
params6 = [
 {'max_depth':[6],
 'min_child_weight':[4],
 'gamma':[0.1],
 'subsample':[0.8, 0.9],
 'colsample_bytree':[0.5, 0.6],
 'reg_alpha':[1], 
 'objective': ['binary:logistic'],
 'scale_pos_weight':[1],
 'learning_rate': [0.1],
 'n_estimators': [1000]}
]

In [33]:
grid6 = GridSearchCV(xgb.XGBClassifier(), param_grid = params6, scoring='roc_auc', n_jobs=-1, iid=False, cv=3, verbose = 2)
grid6.fit(X_train, y_train.ravel())

Fitting 3 folds for each of 4 candidates, totalling 12 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:  2.4min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:  2.4min finished


GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=3, min_child_weight=1, missing=None,
       n_estimators=100, n_jobs=1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
       subsample=1, verbosity=1),
       fit_params=None, iid=False, n_jobs=-1,
       param_grid=[{'max_depth': [6], 'min_child_weight': [4], 'gamma': [0.1], 'subsample': [0.8, 0.9], 'colsample_bytree': [0.5, 0.6], 'reg_alpha': [1], 'objective': ['binary:logistic'], 'scale_pos_weight': [1], 'learning_rate': [0.1], 'n_estimators': [1000]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=2)

In [34]:
xgb6 = grid6.best_estimator_
print(xgb6.get_params()['subsample'])
print(xgb6.get_params()['colsample_bytree'])

0.8
0.6


In [35]:
params7 = [
 {'max_depth':[6],
 'min_child_weight':[4],
 'gamma':[0.1],
 'subsample':[0.8],
 'colsample_bytree':[0.6],
 'reg_alpha':[0.5, 1, 1.5], 
 'objective': ['binary:logistic'],
 'scale_pos_weight':[1],
 'learning_rate': [0.1],
 'n_estimators': [1000]}
]

In [36]:
grid7 = GridSearchCV(xgb.XGBClassifier(), param_grid = params7, scoring='roc_auc', n_jobs=-1, iid=False, cv=3, verbose = 2)
grid7.fit(X_train, y_train.ravel())

Fitting 3 folds for each of 2 candidates, totalling 6 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   6 | elapsed:   48.9s remaining:   48.9s
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:  1.4min finished


GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=3, min_child_weight=1, missing=None,
       n_estimators=100, n_jobs=1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
       subsample=1, verbosity=1),
       fit_params=None, iid=False, n_jobs=-1,
       param_grid=[{'max_depth': [6], 'min_child_weight': [4], 'gamma': [0.1], 'subsample': [0.8], 'colsample_bytree': [0.6], 'reg_alpha': [0.5, 1], 'objective': ['binary:logistic'], 'scale_pos_weight': [1], 'learning_rate': [0.1], 'n_estimators': [1000]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=2)

In [37]:
xgb7 = grid7.best_estimator_
print(xgb7.get_params()['reg_alpha'])

1


In [38]:
params8 = [
 {'max_depth':[6],
 'min_child_weight':[4],
 'gamma':[0.1],
 'subsample':[0.8],
 'colsample_bytree':[0.6],
 'reg_alpha':[1], 
 'objective': ['binary:logistic'],
 'scale_pos_weight':[1],
 'learning_rate': [0.05, 0.1],
 'n_estimators': [1000]}
]

In [39]:
grid8 = GridSearchCV(xgb.XGBClassifier(), param_grid = params8, scoring='roc_auc', n_jobs=-1, iid=False, cv=3, verbose = 2)
grid8.fit(X_train, y_train.ravel())

Fitting 3 folds for each of 2 candidates, totalling 6 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   6 | elapsed:  1.0min remaining:  1.0min
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:  1.6min finished


GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=3, min_child_weight=1, missing=None,
       n_estimators=100, n_jobs=1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
       subsample=1, verbosity=1),
       fit_params=None, iid=False, n_jobs=-1,
       param_grid=[{'max_depth': [6], 'min_child_weight': [4], 'gamma': [0.1], 'subsample': [0.8], 'colsample_bytree': [0.6], 'reg_alpha': [1], 'objective': ['binary:logistic'], 'scale_pos_weight': [1], 'learning_rate': [0.05, 0.1], 'n_estimators': [1000]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=2)

In [40]:
xgb8 = grid8.best_estimator_
print(xgb8.get_params()['learning_rate'])

0.05


In [42]:
params9 = [
 {'max_depth':[6],
 'min_child_weight':[4],
 'gamma':[0.1],
 'subsample':[0.8],
 'colsample_bytree':[0.6],
 'reg_alpha':[1], 
 'objective': ['binary:logistic'],
 'scale_pos_weight':[1],
 'learning_rate': [0.05],
 'n_estimators': [500, 1000]}
]

In [43]:
grid9 = GridSearchCV(xgb.XGBClassifier(), param_grid = params9, scoring='roc_auc', n_jobs=-1, iid=False, cv=3, verbose = 2)
grid9.fit(X_train, y_train.ravel())

Fitting 3 folds for each of 2 candidates, totalling 6 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   6 | elapsed:   29.5s remaining:   29.5s
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:  1.1min finished


GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=3, min_child_weight=1, missing=None,
       n_estimators=100, n_jobs=1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
       subsample=1, verbosity=1),
       fit_params=None, iid=False, n_jobs=-1,
       param_grid=[{'max_depth': [6], 'min_child_weight': [4], 'gamma': [0.1], 'subsample': [0.8], 'colsample_bytree': [0.6], 'reg_alpha': [1], 'objective': ['binary:logistic'], 'scale_pos_weight': [1], 'learning_rate': [0.05], 'n_estimators': [500, 1000]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=2)

In [44]:
xgb9 = grid9.best_estimator_
print(xgb9.get_params()['n_estimators'])

1000


In [46]:
xgb9.fit(X_train, y_train.ravel())
print('Accuracy: %f' % metrics.accuracy_score(y_test, xgb9.predict(X_test)))
print('AUC: %f' %metrics.roc_auc_score(y_test, xgb9.predict(X_test)))

Accuracy: 0.963009
AUC: 0.726545


In [47]:
y_submission23 = pd.DataFrame(xgb9.predict_proba(X_pred)[:, 1], columns = ['Y']) 
y_submission23['Id'] = X_pred_data['Id']
y_submission23 = y_submission23.reindex(columns=["Id", "Y"])
y_submission23.to_csv("submission23_uk734.csv", index=False)

In [53]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [1]:
xgb10 = grid9.best_estimator_
xgb10.fit(X_train, y_train.ravel())
print('Accuracy: %f' % metrics.accuracy_score(y_test, xgb10.predict(X_test)))
print('AUC: %f' %metrics.roc_auc_score(y_test, xgb10.predict_proba(X_test)[:, 1]))

NameError: name 'grid9' is not defined

In [55]:
y_submission24 = pd.DataFrame(xgb10.predict_proba(X_pred)[:, 1], columns = ['Y']) 
y_submission24['Id'] = X_pred_data['Id']
y_submission24 = y_submission24.reindex(columns=["Id", "Y"])
y_submission24.to_csv("submission24_uk734.csv", index=False)

For some reason, matolotlib doesn't work well with xgboost.